In [18]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


In [19]:
dataset = pd.read_csv('data/healthcare-dataset-stroke-data.csv' , sep=',')

dataset = dataset.drop(dataset.columns[0], axis=1)

dataset = dataset.dropna()

dataset.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4909 non-null   object 
 1   age                4909 non-null   float64
 2   hypertension       4909 non-null   int64  
 3   heart_disease      4909 non-null   int64  
 4   ever_married       4909 non-null   object 
 5   work_type          4909 non-null   object 
 6   Residence_type     4909 non-null   object 
 7   avg_glucose_level  4909 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     4909 non-null   object 
 10  stroke             4909 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 460.2+ KB


In [20]:
dataset["gender"] = dataset["gender"].replace(["Other"], "Female")
gender_conversion = {"Male": 0, "Female": 1}
dataset["gender"] = dataset["gender"].map(gender_conversion)
dataset["gender"] = dataset["gender"].astype(int)


In [21]:
married_conversion = {"No": 0, "Yes": 1}
dataset["ever_married"] = dataset["ever_married"].map(married_conversion)
dataset["ever_married"] = dataset["ever_married"].astype(int)


In [22]:
df_work_ohe = pd.get_dummies(
    dataset["work_type"], 
    prefix="work_ohe", 
    drop_first=True,
)
dataset = pd.concat([dataset, df_work_ohe], axis=1)
dataset = dataset.drop(["work_type"], axis=1)


In [23]:
residence_conversion = {"Rural": 0, "Urban": 1}
dataset["Residence_type"] = dataset["Residence_type"].map(residence_conversion)
dataset["Residence_type"] = dataset["Residence_type"].astype(int)


In [24]:
df_smoking_ohe = pd.get_dummies(
    dataset["smoking_status"], 
    prefix="smoking_ohe", 
    drop_first=True,
)
df_smoking_ohe = df_smoking_ohe.rename(columns={
    "smoking_ohe_never smoked": "smoking_ohe_never_smoked", 
    "smoking_ohe_formerly smoked": "smoking_ohe_formerly_smoked",
})
dataset = pd.concat([dataset, df_smoking_ohe], axis=1)
dataset = dataset.drop(["smoking_status"], axis=1)


In [25]:
dataset.head()

,gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,stroke,work_ohe_Never_worked,work_ohe_Private,work_ohe_Self-employed,work_ohe_children,smoking_ohe_formerly_smoked,smoking_ohe_never_smoked,smoking_ohe_smokes
0,0,67.0,0,1,1,1,228.69,36.6,1,0,1,0,0,1,0,0
2,0,80.0,0,1,1,0,105.92,32.5,1,0,1,0,0,0,1,0
3,1,49.0,0,0,1,1,171.23,34.4,1,0,1,0,0,0,0,1
4,1,79.0,1,0,1,0,174.12,24.0,1,0,0,1,0,0,1,0
5,0,81.0,0,0,1,1,186.21,29.0,1,0,1,0,0,1,0,0


In [26]:
x = dataset.iloc[:, 0:10].values
y = dataset.iloc[:, 10].values


In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)


In [28]:
model = Sequential()
model.add(Dense(12, input_dim=10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='he_normal'))


In [29]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12)                132       
                                                                 
 dense_4 (Dense)             (None, 8)                 104       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 245
Trainable params: 245
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [31]:
model.fit(x_train, y_train, epochs=150, batch_size=10)


Epoch 1/150
344/344 [==============================] - 1s 1ms/step - loss: 0.8523 - accuracy: 0.6007
Epoch 2/150
344/344 [==============================] - 1s 2ms/step - loss: 0.6414 - accuracy: 0.6502
Epoch 3/150
344/344 [==============================] - 1s 2ms/step - loss: 0.6087 - accuracy: 0.6729
Epoch 4/150
344/344 [==============================] - 0s 1ms/step - loss: 0.5942 - accuracy: 0.6848
Epoch 5/150
344/344 [==============================] - 1s 2ms/step - loss: 0.5879 - accuracy: 0.6825
Epoch 6/150
344/344 [==============================] - 1s 1ms/step - loss: 0.5929 - accuracy: 0.6848
Epoch 7/150
344/344 [==============================] - 1s 2ms/step - loss: 0.5819 - accuracy: 0.6868
Epoch 8/150
344/344 [==============================] - 1s 1ms/step - loss: 0.5818 - accuracy: 0.6883
Epoch 9/150
344/344 [==============================] - 1s 2ms/step - loss: 0.5815 - accuracy: 0.6831
Epoch 10/150
344/344 [==============================] - 0s 1ms/step - loss: 0.5888 - accura

In [32]:
loss, accuracy = model.evaluate(x_test, y_test)
print("\nLoss: %.2f, Acurácia: %.2f%%" % (loss, accuracy*100))

47/47 [==============================] - 0s 1ms/step - loss: 0.5544 - accuracy: 0.7013

Loss: 0.55, Acurácia: 70.13%


In [33]:
predictions = model.predict(x)

154/154 [==============================] - 0s 1ms/step


In [34]:
# Ajusta as previsões e imprime o resultado
previsões = [round(x[0]) for x in predictions]
print(previsões)

[1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 